# Confirming the Anatomical Region of the Ablation Using Freesurfer LUT

After annotating the ablation region of the post surgical MRI with Seg3D, we use Slicer to reorient the MRI from RAS to freesurfer (FS) space. Then on the mask, we overlay the Desikan-Killiany atlas provided by Freesurfer, matching each voxel to its corresponding brain region in the atlas. Compare the resulting list of brain regions to what was clinically reported will allow us to confirm that the mask was mapped to the correct space. 

We will us la02 as an example.

In [1]:
# import modules
import os
import nibabel as nb
import numpy as np
import nrrd
import collections

Freesurfer's lookup table (LUT) assigns a number to a certain brain region. The full table can be found here:
https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/AnatomicalROI/FreeSurferColorLUT

In [2]:
# get labels using Freesurfer's lookup table (LUT)
fs_lut_fpath = "C:\\Users\\d0156\\Dropbox\\bids_layout_data\\sub-la02\\FreeSurferColorLUT.txt"
fid = open(fs_lut_fpath)
LUT = fid.readlines()
fid.close()

# make dictionary of labels
LUT = [row.split() for row in LUT]
lab = {}
for row in LUT:
    if (
        len(row) > 1 and row[0][0] is not "#" and row[0][0] is not "\\"
    ):
        lname = row[1]
        lab[np.int(row[0])] = lname

print("Loading lookup table for freesurfer labels")

Loading lookup table for freesurfer labels


Freesurfer provides three different types of atlases: Desikan-Killiany, DKT, and Destrieux. The three are trained in different ways, but in general, an atlas is a model of the cortical surface based on probabilistic information estimated from a manually labeled training set. For more information, visit
https://surfer.nmr.mgh.harvard.edu/fswiki/CorticalParcellation

In [3]:
# assuming atlas type is desikan-killiany
depth_atlas_suffix = ""
mri_dir = "C:\\Users\\d0156\\Dropbox\\bids_layout_data\\la02\\mri"

# load in ASEG image file from atlas
aseg_fpath = os.path.join(mri_dir, "aparc%s+aseg.mgz" % (depth_atlas_suffix))
depth_atlas_img = nb.freesurfer.load(aseg_fpath)
aparc_dat = depth_atlas_img.get_fdata()

print("Loading atlas")

Loading atlas


After annotating the ablation in Seg3D and mapping the mask to FS space using Slicer, we index the annotated voxels of the mask. 

In [8]:
# load mask image
mask_dir = "C:\\Users\\d0156\\Dropbox\\bids_layout_data\\sub-la02\\test"
mask_fpath = os.path.join(mask_dir, "sub-la02_ses-postsurgery_proc-slicer.nii")
mask_img = nb.load(mask_fpath)
mask_data = mask_img.get_fdata()

print("Loading mask")

# determine where mask is
mask_indx = np.argwhere(mask_data)
print(mask_indx)
print("Loading mask index")

Loading mask
[[132 108 187]
 [132 108 188]
 [132 109 185]
 ...
 [147 116 183]
 [147 116 184]
 [147 116 185]]
Loading mask index


We overlay the annotated voxels with the atlas and the LUT, which gives us a list of brain regions within the mask. This should be similar to what was reported clinically. 

In [9]:
# determine list of brain regions in mask
aparc_indx = []
for i in range(len(mask_indx)):
    aparc_indx.append(aparc_dat[mask_indx[i][0], mask_indx[i][1], mask_indx[i][2]])

regions = []
for i in range(len(aparc_indx)):
    if (aparc_indx[i] != 0) and (lab[aparc_indx[i]] not in regions):
        regions.append(lab[aparc_indx[i]])

print("Loading brain regions in mask")

print("Brain regions in mask: ")
print(regions)

Loading brain regions in mask
Brain regions in mask: 
['ctx-lh-superiorfrontal', 'Left-Cerebral-White-Matter', 'WM-hypointensities', 'ctx-lh-rostralmiddlefrontal']


In [10]:
def _from_tsv(fname, dtypes=None):
    """Read a tsv file into an OrderedDict.
    Parameters
    ----------
    fname : str
        Path to the file being loaded.
    dtypes : list, optional
        List of types to cast the values loaded as. This is specified column by
        column.
        Defaults to None. In this case all the data is loaded as strings.
    Returns
    -------
    data_dict : collections.OrderedDict
        Keys are the column names, and values are the column data.
    """
    data = np.loadtxt(fname, dtype=str, delimiter='\t',
                      comments=None, encoding='utf-8')
    column_names = data[0, :]
    info = data[1:, :]
    data_dict = collections.OrderedDict()
    if dtypes is None:
        dtypes = [str] * info.shape[1]
    if not isinstance(dtypes, (list, tuple)):
        dtypes = [dtypes] * info.shape[1]
    if not len(dtypes) == info.shape[1]:
        raise ValueError('dtypes length mismatch. Provided: {0}, '
                         'Expected: {1}'.format(len(dtypes), info.shape[1]))
    for i, name in enumerate(column_names):
        data_dict[name] = info[:, i].astype(dtypes[i]).tolist()
    return data_dict

In [77]:
# load mask image (filled) 
mask_fill_dir = "C:\\Users\\d0156\\Dropbox\\bids_layout_data\\sub-la02"
mask_fill_fpath = os.path.join(mask_fill_dir, "sub-la02_ses-postsurgery_proc-slicerfillaxial.nii.gz")
mask_fill_img = nb.load(mask_fill_fpath)
mask_fill_data = mask_fill_img.get_fdata()

print(mask_fill_data.shape)
print("Loading mask")

# determine where mask is
mask_fill_indx = np.argwhere(mask_fill_data)
print(mask_indx.shape)
print(mask_fill_indx.shape)
print("Loading mask index")

mask_fill_aff = mask_fill_img.affine

# load pret1 image
pret1_fpath = "C:\\Users\\d0156\\Dropbox\\bids_layout_data\\sub-la02\\test\\sub-la02_ses-presurgery_space-fs_T1w.nii"
pret1_img = nb.load(postt1_fpath)
pret1_aff = pret1_img.affine
print(pret1_aff)
print(mask_fill_aff)

(256, 256, 256)
Loading mask
(729, 3)
(2140, 3)
Loading mask index
[[-9.99999940e-01  0.00000000e+00  0.00000000e+00  1.26817993e+02]
 [ 0.00000000e+00 -1.49011612e-08  1.00000000e+00 -1.21573318e+02]
 [ 0.00000000e+00 -1.00000000e+00  0.00000000e+00  1.59645630e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-9.99999932e-01  2.61643042e-04 -2.61643042e-04  1.26817993e+02]
 [-2.61643042e-04  3.42285418e-08  9.99999966e-01 -1.21573318e+02]
 [-2.61643042e-04 -9.99999966e-01 -3.42285418e-08  1.59645630e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [79]:
pre = _from_tsv("C:\\Users\\d0156\\Johns Hopkins\\Adam Li - epilepsy_bids\\sub-la02\\ses-presurgery\\ieeg\\sub-la02_ses-presurgery_acq-seeg_space-fs_electrodes.tsv")

keys = list(pre.keys())
values = list(pre.values())
coords = [[] for c in range(len(values[0]))]

for i in range(len(values[0])):
    for j in range(len(keys)):
        coords[i].append(values[j][i])

elec_coords = np.zeros((len(coords), 3))

for i in range(len(coords)):
    for j in range(3):
        elec_coords[i][j] = float(coords[i][j+1])

### comparing voxels

# convert xyz coordinates of electrodes to voxels of mask
elec_vox = np.floor(nb.affines.apply_affine(np.linalg.inv(pret1_aff), elec_coords))
#elec_vox = np.round(nb.affines.apply_affine(np.linalg.inv(mask_fill_aff), elec_coords))

print(elec_vox)
print(mask_fill_indx)

# compare voxels of electrodes and mask to find overlapping regions (should match clinical results)
elecs = []
elecs_name = []

for i in range(len(mask_fill_indx)):
    for j in range(len(elec_vox)):
        if (np.abs(elec_vox[j][0] - mask_fill_indx[i][0]) <= 3.) & (np.abs(elec_vox[j][1] - mask_fill_indx[i][1]) <= 3.) \
            & (np.abs(elec_vox[j][2] - mask_fill_indx[i][2]) <= 3.):
            if (coords[j][0] not in elecs):
                elecs.append(coords[j][0])
                elecs_name.append(coords[j][5])

print(elecs)
print(elecs_name)

[[127. 124. 188.]
 [130. 124. 189.]
 [134. 124. 189.]
 [137. 123. 188.]
 [140. 123. 190.]
 [144. 123. 190.]
 [147. 123. 191.]
 [151. 122. 191.]
 [161. 121. 192.]
 [165. 121. 194.]
 [168. 121. 194.]
 [172. 120. 194.]
 [126. 103. 176.]
 [129. 102. 177.]
 [133. 102. 177.]
 [136. 101. 177.]
 [139. 100. 177.]
 [143. 100. 178.]
 [146.  99. 178.]
 [154. 122. 191.]
 [158. 122. 192.]
 [150.  98. 178.]
 [153.  98. 179.]
 [157.  97. 179.]
 [160.  96. 180.]
 [164.  94. 181.]
 [125. 138. 185.]
 [130. 138. 186.]
 [133. 137. 187.]
 [136. 137. 186.]
 [140. 137. 187.]
 [144. 137. 186.]
 [147. 137. 187.]
 [151. 136. 188.]
 [154. 137. 189.]
 [158. 137. 189.]
 [161. 137. 189.]
 [165. 137. 190.]
 [169. 137. 191.]
 [172. 137. 191.]
 [129. 153. 165.]
 [132. 152. 166.]
 [136. 151. 166.]
 [139. 151. 166.]
 [142. 150. 167.]
 [146. 149. 167.]
 [150. 148. 167.]
 [153. 148. 168.]
 [157. 147. 168.]
 [160. 146. 168.]
 [163. 146. 169.]
 [167. 145. 169.]
 [170. 145. 170.]
 [173. 144. 170.]
 [177. 143. 170.]
 [179. 143

In [72]:
### comparing xyz coordinates

# convert voxels of mask to xyz coordinates of electrodes
mask_coords = nb.affines.apply_affine(mask_fill_aff, mask_fill_indx)

# compare xyz coordinates of electrodes and mask to find overlapping regions (should match clinical results)
elecs = []
elecs_name = []

for i in range(len(mask_coords)):
    for j in range(len(elec_coords)):
        if (np.abs(elec_coords[j][0] - mask_coords[i][0]) <= 4) & (np.abs(elec_coords[j][1] - mask_coords[i][1]) <= 4) \
            & (np.abs(elec_coords[j][2] - mask_coords[i][2]) <= 4):
            if (coords[j][0] not in elecs):
                elecs.append(coords[j][0])
                elecs_name.append(coords[j][5])

print(elecs)
print(elecs_name)

["L'4", "L'5", "G'6", "L'6"]
['Left-Cerebral-White-Matter', 'ctx-lh-superiorfrontal', 'Left-Cerebral-White-Matter', 'Left-Cerebral-White-Matter']
